# Embeddings y Procesamiento de Texto - LLMs desde Cero

El objetivo de este notebook es entender cómo los modelos de lenguaje convierten texto en números, y luego esos números en vectores que capturan significado.

Vamos a seguir paso a paso el pipeline: tokenización → fragmentos → embeddings → similaridad.

## 1. Importar librerías y cargar el texto

In [ ]:
import torch
import tiktoken
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el texto
with open('the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

print(f"Archivo cargado: {len(raw_text)} caracteres")
print(f"Primeros 300 caracteres: {raw_text[:300]}...")

## 2. Tokenización: Convertir texto a números

Los LLMs no entienden palabras, entienden tokens. tiktoken usa BPE (Byte-Pair Encoding), igual que GPT-2. Cada token es un número entre 0 y 50,256. Palabras comunes son un token, palabras raras pueden ser varios.

In [ ]:
# Tokenizar con GPT-2
tokenizer = tiktoken.get_encoding('gpt2')
token_ids = tokenizer.encode(raw_text)

print(f"Total de tokens: {len(token_ids)}")
print(f"Vocab size (aprox): {tokenizer.n_vocab}")
print(f"Primeros 20 tokens: {token_ids[:20]}")

# Ver qué dice el primer fragmento
sample = token_ids[:10]
decoded = tokenizer.decode(sample)
print(f"Primeros tokens decodificados: '{decoded}'")

## 3. Ventana deslizante: Crear contextos

Para entrenar un modelo, necesitamos pares (entrada, siguiente_token). Usamos una ventana deslizante que se mueve sobre el texto. El tamaño de la ventana (`max_length`) y cuánto se mueve (`stride`) controlan cuántos ejemplos generamos.

Simulemos distintas configuraciones para ver el efecto del overlap.

In [ ]:
class DatasetLLM(Dataset):
    """Crea pares (input_tokens, target_token) con ventana deslizante."""
    def __init__(self, token_ids, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        
        # Ventana deslizante
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1:i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunk, dtype=torch.long))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


# Experimento: variar max_length y stride
configs = [
    (32, 32),
    (32, 16),
    (64, 16),
    (128, 64),
    (128, 32),
    (256, 128)
]

print("Experimento: efecto del solapamiento en la cantidad de datos\n")
for max_len, st in configs:
    dataset = DatasetLLM(token_ids, max_len, st)
    overlap_pct = ((max_len - st) / max_len) * 100 if max_len > st else 0
    print(f"max_length={max_len:3d}, stride={st:3d} → {len(dataset):4d} muestras | overlap: {overlap_pct:5.1f}%")

print("\nObservación: stride pequeño = más solapamiento = más datos.")
print("Pero también más redundancia. Hay que encontrar el balance.")

## 4. Capa de Embeddings: Del número al vector

Los embeddings son como una tabla de búsqueda gigante. Cada token (número) mapea a un vector de N dimensiones. En GPT-2, son vectores de 768 dimensiones.

La pregunta clave: **¿Por qué estos vectores capturan significado?**

Respuesta corta: Porque durante el entrenamiento, el modelo ajusta estos vectores mediante backprop para predecir el siguiente token. Tokens que aparecen en contextos similares terminan con vectores similares. Eso es lo que SÍ es significado: tokens que se usan de formas parecidas son similares.

In [ ]:
# Crear una capa de embeddings
vocab_size = 50257  # Tamaño del vocabulario de GPT-2
embedding_dim = 768  # Dimensión de los embeddings en GPT-2

embedding_layer = torch.nn.Embedding(vocab_size, embedding_dim)

print(f"Capa de embeddings creada")
print(f"  - Input: índices de tokens (0 a {vocab_size-1})")
print(f"  - Output: vectores de {embedding_dim} dimensiones")
print(f"  - Parámetros totales: {vocab_size * embedding_dim:,}")

# Generar embeddings para una muestra
dataset_sample = DatasetLLM(token_ids, max_length=64, stride=32)
loader = DataLoader(dataset_sample, batch_size=4)

# Tomar un batch
batch_inputs, batch_targets = next(iter(loader))
print(f"\nBatch de entrada shape: {batch_inputs.shape}")
print(f"  - {batch_inputs.shape[0]} ejemplos, cada uno con {batch_inputs.shape[1]} tokens")

# Generar embeddings
embeddings = embedding_layer(batch_inputs)
print(f"\nEmbeddings generados: {embeddings.shape}")
print(f"  - {embeddings.shape[0]} ejemplos")
print(f"  - {embeddings.shape[1]} posiciones (tokens en el contexto)")
print(f"  - {embeddings.shape[2]} dimensiones (el vector)")

## 5. Similaridad: Los vectores sí capturan estructura

Una forma de "entender" qué captura un embedding es mirar qué tan similares son según distance coseno. Dos tokens que generalmente aparecen en contextos parecidos tendrán embeddings parecidos.

In [ ]:
# Tomar un embedding del batch anterior
sample_embeddings = embeddings[0].detach().numpy()  # Shape: (64, 768)

# Matriz de similaridad coseno entre los 64 tokens del contexto
similarity_matrix = cosine_similarity(sample_embeddings)

print(f"Matriz de similaridad: {similarity_matrix.shape}")
print(f"")
print(f"Valores en la diagonal (token consigo mismo): ~1.0")
print(f"Diagonal medio: {np.diag(similarity_matrix).mean():.3f}")
print(f"")
print(f"Valores típicos fuera de diagonal (tokens distintos):")
off_diagonal = similarity_matrix[np.triu_indices_from(similarity_matrix, k=1)]
print(f"  Min: {off_diagonal.min():.3f}")
print(f"  Mean: {off_diagonal.mean():.3f}")
print(f"  Max: {off_diagonal.max():.3f}")
print(f"")
print(f"Por ahora los embeddings son aleatorios (no entrenados),")
print(f"pero la estructura está lista para cuando se entrene.")

## 6. Reflexión: Por qué importa esto

Este flujo completo (tokenización → chunking → embeddings → comparación) es la base de:

- **Recuperación semántica**: Encontrar contextos similares rápidamente (RAG)
- **Razonamiento multi-paso**: Los agentes navegan usando similaridad vectorial
- **Memoria**: Guardar y recuperar información según relevancia

A partir de aquí, el siguiente paso son los **transformers**: agregar atención ("¿cuál de estos tokens es importante?") y más capas. Pero la base son los embeddings.

In [ ]:
# Resumen estadístico
print("=" * 60)
print("RESUMEN DEL FLUJO")
print("=" * 60)

dataset_default = DatasetLLM(token_ids, max_length=128, stride=64)

print(f"\n1. TEXTO ORIGINAL:")
print(f"   - {len(raw_text)} caracteres")
print(f"   - {len(token_ids)} tokens después de BPE")

print(f"\n2. TOKENIZACIÓN:")
print(f"   - Vocabulario GPT-2: 50,257 tokens")
print(f"   - Encoding: BPE (Byte-Pair Encoding)")

print(f"\n3. DATASET (con max_length=128, stride=64):")
print(f"   - Muestras generadas: {len(dataset_default)}")
print(f"   - Cada muestra: 128 tokens entrada + 128 tokens target")
print(f"   - Solapamiento: 50%")

print(f"\n4. EMBEDDINGS:")
print(f"   - Dimensión: 768 (como GPT-2)")
print(f"   - Parámetros por token: 768")
print(f"   - Parámetros totales capa: {vocab_size * embedding_dim:,}")

print(f"\n5. SIMILITUD:")
print(f"   - Métrica: Coseno entre vectores")
print(f"   - Rango: [-1, 1] típicamente [0, 1]")

print("\n" + "=" * 60)
print("El código está listo para:")
print("  - Fine-tuning: entrenar estos embeddings con backprop")
print("  - Agentes: usar similaridad para recuperación y razonamiento")
print("  - Transformers: agregar capas de atención encima")
print("=" * 60)